# 🏗️ Extraction de Plans Numériques depuis PDF d'Architecte

## 📋 Projet: Extraction Automatique de Plans Numériques Exploitables

**Objectif**: Créer un outil d'extraction automatique de plans numériques exploitables (format vectoriel ou BIM) à partir de documents PDF d'architecte grâce à la vision par ordinateur et au deep learning.

**Livrable**: Notebook unique et complet avec pipeline end-to-end

---

## 🚀 Guide Rapide d'Utilisation

### ⚡ Démarrage Rapide (3 étapes)

```python
# 1. Exécuter Section 1 (Setup) - OBLIGATOIRE
# 2. Placer vos PDFs dans data/pdfs/
# 3. Exécuter ce code:

pdf_path = "data/pdfs/mon_plan.pdf"
result = process_pdf(pdf_path)  # Extraction
annotations = labeler.label_image(result['saved_paths'][0])  # Détection
labeler.visualize_annotations(result['saved_paths'][0], annotations)  # Visualiser
plt.show()
```

### 📖 Documentation Complète

Voir le fichier **`GUIDE_UTILISATION.md`** pour:
- Installation détaillée
- Configuration complète
- Workflow étape par étape
- Détection et extraction
- Visualisation des résultats
- Dépannage

---

## 🎯 Architecture du Pipeline

```
PDF Input → Extraction Images → Preprocessing → Détection (YOLO) → Segmentation → 
Post-processing → Export Vectoriel/BIM → Visualisation → Stockage Snowflake
```

---

## 📚 Structure du Notebook

1. **Setup et Imports** - Configuration environnement ⚠️ OBLIGATOIRE
2. **Snowflake Configuration** - Connexion et schémas (optionnel)
3. **Extraction PDF** - Conversion PDF → Images ⚠️ OBLIGATOIRE
4. **Génération Données LLM** - Augmentation avec IA générative (optionnel)
5. **Labellisation** - Automatique et manuelle ⚠️ OBLIGATOIRE
6. **Entraînement YOLO** - Fine-tuning modèles (optionnel, long)
7. **Inférence** - Extraction sur nouveaux PDF ⚠️ OBLIGATOIRE
8. **Visualisation** - Dashboard interactif ⚠️ OBLIGATOIRE
9. **Tests et Validation** - Métriques et qualité (recommandé)


# Section 1: Setup et Imports

## 1.1 Installation des dépendances (à exécuter une seule fois)


## 1.4 Téléchargement des Modèles YOLO (Optionnel - Première Utilisation)

Les modèles YOLO seront téléchargés automatiquement lors de leur première utilisation, mais vous pouvez les pré-télécharger ici pour éviter les attentes.


In [2]:
# ============================================================================
# TÉLÉCHARGEMENT DES MODÈLES YOLO
# ============================================================================
# 
# IMPORTANT: Les modèles seront téléchargés automatiquement lors de leur 
# première utilisation dans la Section 5. Cette cellule est optionnelle 
# mais recommandée pour pré-télécharger les modèles.
#
# Méthodes alternatives:
# 1. Exécuter cette cellule dans le notebook (RECOMMANDÉ)
# 2. Créer un script Python séparé (voir ci-dessous)
# 3. Utiliser PowerShell/terminal (voir ci-dessous)
# ============================================================================

print("🔄 Téléchargement des modèles YOLO...")
print("⏳ Cela peut prendre quelques minutes lors du premier téléchargement...\n")

try:
    # Modèle détection (le plus utilisé)
    print("📥 Téléchargement: yolo11n.pt (Détection)...")
    model_det = YOLO("yolo11n.pt")
    print("   ✅ Modèle détection téléchargé\n")
    
    # Modèle segmentation
    print("📥 Téléchargement: yolo11n-seg.pt (Segmentation)...")
    model_seg = YOLO("yolo11n-seg.pt")
    print("   ✅ Modèle segmentation téléchargé\n")
    
    # Modèle keypoints
    print("📥 Téléchargement: yolo11n-pose.pt (Keypoints)...")
    model_kpt = YOLO("yolo11n-pose.pt")
    print("   ✅ Modèle keypoints téléchargé\n")
    
    print("=" * 60)
    print("✅ TOUS LES MODÈLES SONT TÉLÉCHARGÉS!")
    print("=" * 60)
    print("\n💡 Les modèles sont sauvegardés dans:")
    print(f"   {Path.home() / '.ultralytics'}")
    
except Exception as e:
    print(f"❌ Erreur lors du téléchargement: {e}")
    print("\n💡 Les modèles seront téléchargés automatiquement lors de leur première utilisation")
    print("   Vous pouvez continuer avec le notebook, le téléchargement se fera dans la Section 5")


🔄 Téléchargement des modèles YOLO...
⏳ Cela peut prendre quelques minutes lors du premier téléchargement...

📥 Téléchargement: yolo11n.pt (Détection)...
❌ Erreur lors du téléchargement: name 'YOLO' is not defined

💡 Les modèles seront téléchargés automatiquement lors de leur première utilisation
   Vous pouvez continuer avec le notebook, le téléchargement se fera dans la Section 5


### 📝 Méthodes Alternatives de Téléchargement

Si vous préférez télécharger les modèles en dehors du notebook:

**Option 1: Script Python séparé**
```bash
# Créer un fichier download_models.py
# Puis exécuter:
python download_models.py
```

**Option 2: PowerShell/CMD (Windows)**
```powershell
# Activer l'environnement virtuel d'abord
venv\Scripts\activate

# Puis exécuter Python directement
python -c "from ultralytics import YOLO; YOLO('yolo11n.pt'); YOLO('yolo11n-seg.pt'); YOLO('yolo11n-pose.pt')"
```

**Option 3: Terminal Linux/Mac**
```bash
# Activer l'environnement virtuel
source venv/bin/activate

# Exécuter
python -c "from ultralytics import YOLO; YOLO('yolo11n.pt'); YOLO('yolo11n-seg.pt'); YOLO('yolo11n-pose.pt')"
```

**Note**: Les modèles seront téléchargés automatiquement lors de leur première utilisation dans la Section 5 si vous ne les téléchargez pas maintenant.


In [3]:
# Installation des packages nécessaires (décommenter si nécessaire)
!pip install opencv-python pillow scikit-image ultralytics torch torchvision
!pip install PyMuPDF pdf2image pdfplumber
!pip install openai anthropic transformers langchain
!pip install diffusers controlnet-aux
!pip install snowflake-connector-python snowflake-sqlalchemy
!pip install matplotlib plotly seaborn
!pip install ezdxf ifcopenshell
!pip install pandas numpy scipy
!pip install ipywidgets



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1.2 Imports standards


In [ ]:
# Imports standards
import os
import sys
import json
import warnings
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Tuple, Optional
import base64
from io import BytesIO

# Data manipulation
import pandas as pd
import numpy as np

# Computer Vision
import cv2
from PIL import Image, ImageDraw, ImageFont
import skimage
from skimage import filters, morphology, transform

# PDF Processing
import fitz  # PyMuPDF
from pdf2image import convert_from_path
import pdfplumber

# Deep Learning
import torch
import torchvision
from ultralytics import YOLO
from ultralytics.utils import LOGGER

# LLM and Generative AI
try:
    from anthropic import Anthropic
    ANTHROPIC_AVAILABLE = True
except ImportError:
    ANTHROPIC_AVAILABLE = False
    print("Anthropic non disponible, utiliser un modèle local")

try:
    from transformers import pipeline
    TRANSFORMERS_AVAILABLE = True
except ImportError:
    TRANSFORMERS_AVAILABLE = False

# Snowflake
try:
    import snowflake.connector
    from snowflake.connector.pandas_tools import write_pandas
    SNOWFLAKE_AVAILABLE = True
except ImportError:
    SNOWFLAKE_AVAILABLE = False
    print("Snowflake non disponible")

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns

# Export formats
try:
    import ezdxf
    EZDXF_AVAILABLE = True
except ImportError:
    EZDXF_AVAILABLE = False

try:
    import ifcopenshell
    IFC_AVAILABLE = True
except ImportError:
    IFC_AVAILABLE = False

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Tous les imports sont réussis!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")


✅ Tous les imports sont réussis!
PyTorch version: 2.9.1+cpu
CUDA disponible: False


## 1.3 Configuration globale et variables


In [ ]:
# ============================================================================
# CONFIGURATION GLOBALE
# ============================================================================

class Config:
    """Configuration centrale du projet"""
    
    # Chemins
    BASE_DIR = Path(".")
    DATA_DIR = BASE_DIR / "data"
    PDF_DIR = DATA_DIR / "pdfs"
    IMAGES_DIR = DATA_DIR / "images"
    ANNOTATIONS_DIR = DATA_DIR / "annotations"
    MODELS_DIR = BASE_DIR / "models"
    OUTPUT_DIR = BASE_DIR / "output"
    
    # Création des répertoires
    for dir_path in [DATA_DIR, PDF_DIR, IMAGES_DIR, ANNOTATIONS_DIR, MODELS_DIR, OUTPUT_DIR]:
        dir_path.mkdir(parents=True, exist_ok=True)
    
    # Paramètres PDF
    PDF_DPI = 300  # Résolution pour conversion PDF → Image
    PDF_SCALE = 2.0  # Facteur d'échelle
    
    # Paramètres preprocessing
    IMAGE_SIZE = (1024, 1024)  # Taille standardisée
    BINARY_THRESHOLD = 127  # Seuil binarisation
    
    # Classes YOLO (éléments architecturaux)
    CLASSES = {
        0: "porte",
        1: "fenetre",
        2: "mur",
        3: "escalier",
        4: "colonne",
        5: "sanitaire",
        6: "cuisine",
        7: "couloir",
        8: "piece",
        9: "balcon"
    }
    
    # Paramètres YOLO
    YOLO_MODEL_DETECTION = "yolo11n.pt"  # Modèle pré-entraîné détection
    YOLO_MODEL_SEGMENTATION = "yolo11n-seg.pt"  # Modèle segmentation
    YOLO_MODEL_KEYPOINTS = "yolo11n-pose.pt"  # Modèle keypoints
    YOLO_IMG_SIZE = 640
    YOLO_CONF_THRESHOLD = 0.25
    YOLO_IOU_THRESHOLD = 0.45
    
    # Paramètres LLM
    LLM_MODEL = "claude-3-5-sonnet-20241022"  # Anthropic Claude par défaut
    LLM_TEMPERATURE = 0.7
    NUM_SCENARIOS = 50  # Nombre de scénarios à générer
    
    # Paramètres Snowflake
    SNOWFLAKE_ACCOUNT = os.getenv("SNOWFLAKE_ACCOUNT", "")
    SNOWFLAKE_USER = os.getenv("SNOWFLAKE_USER", "")
    SNOWFLAKE_PASSWORD = os.getenv("SNOWFLAKE_PASSWORD", "")
    SNOWFLAKE_WAREHOUSE = os.getenv("SNOWFLAKE_WAREHOUSE", "COMPUTE_WH")
    SNOWFLAKE_DATABASE = os.getenv("SNOWFLAKE_DATABASE", "ARCHITECTURE_DB")
    SNOWFLAKE_SCHEMA = os.getenv("SNOWFLAKE_SCHEMA", "PLANS_SCHEMA")
    
    # Paramètres génération données
    AUGMENTATION_ROTATIONS = [90, 180, 270]
    AUGMENTATION_FLIP = True
    AUGMENTATION_SCALE = [0.8, 1.0, 1.2]
    AUGMENTATION_NOISE = True

# Instance globale de configuration
config = Config()

print("✅ Configuration initialisée!")
print(f"📁 Répertoires créés: {[str(d) for d in [config.DATA_DIR, config.MODELS_DIR, config.OUTPUT_DIR]]}")


✅ Configuration initialisée!
📁 Répertoires créés: ['data', 'models', 'output']


In [6]:
# ============================================================================
# CONNEXION SNOWFLAKE
# ============================================================================

class SnowflakeManager:
    """Gestionnaire de connexion et opérations Snowflake"""
    
    def __init__(self, config):
        self.config = config
        self.conn = None
        self.cursor = None
        
    def connect(self):
        """Établit la connexion à Snowflake"""
        if not SNOWFLAKE_AVAILABLE:
            print("⚠️ Snowflake non disponible, mode simulation activé")
            return False
            
        try:
            self.conn = snowflake.connector.connect(
                user=self.config.SNOWFLAKE_USER,
                password=self.config.SNOWFLAKE_PASSWORD,
                account=self.config.SNOWFLAKE_ACCOUNT,
                warehouse=self.config.SNOWFLAKE_WAREHOUSE,
                database=self.config.SNOWFLAKE_DATABASE,
                schema=self.config.SNOWFLAKE_SCHEMA
            )
            self.cursor = self.conn.cursor()
            print("✅ Connexion Snowflake réussie!")
            return True
        except Exception as e:
            print(f"❌ Erreur connexion Snowflake: {e}")
            print("💡 Mode simulation activé (pas de connexion réelle)")
            return False
    
    def execute_query(self, query: str, fetch: bool = True):
        """Exécute une requête SQL"""
        if not self.conn:
            print(f"⚠️ Mode simulation - Requête: {query[:100]}...")
            return None
        try:
            self.cursor.execute(query)
            if fetch:
                return self.cursor.fetchall()
            return True
        except Exception as e:
            print(f"❌ Erreur requête: {e}")
            return None
    
    def create_schema(self):
        """Crée le schéma et les tables nécessaires"""
        queries = [
            f"CREATE DATABASE IF NOT EXISTS {self.config.SNOWFLAKE_DATABASE}",
            f"USE DATABASE {self.config.SNOWFLAKE_DATABASE}",
            f"CREATE SCHEMA IF NOT EXISTS {self.config.SNOWFLAKE_SCHEMA}",
            f"USE SCHEMA {self.config.SNOWFLAKE_SCHEMA}",
            """
            CREATE TABLE IF NOT EXISTS plans_metadata (
                id VARCHAR(255) PRIMARY KEY,
                nom_fichier VARCHAR(500),
                date_upload TIMESTAMP,
                dimensions_width INT,
                dimensions_height INT,
                nombre_pages INT,
                taille_fichier_kb FLOAT,
                metadata_json VARIANT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
            )
            """,
            """
            CREATE TABLE IF NOT EXISTS annotations (
                id VARCHAR(255) PRIMARY KEY,
                plan_id VARCHAR(255),
                classe VARCHAR(50),
                bbox_x FLOAT,
                bbox_y FLOAT,
                bbox_width FLOAT,
                bbox_height FLOAT,
                confidence FLOAT,
                segmentation_points VARIANT,
                keypoints VARIANT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP(),
                FOREIGN KEY (plan_id) REFERENCES plans_metadata(id)
            )
            """,
            """
            CREATE TABLE IF NOT EXISTS model_versions (
                id VARCHAR(255) PRIMARY KEY,
                version VARCHAR(50),
                model_type VARCHAR(50),
                mAP FLOAT,
                precision FLOAT,
                recall FLOAT,
                date_entrainement TIMESTAMP,
                chemin_modele VARCHAR(500),
                hyperparameters VARIANT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
            )
            """,
            """
            CREATE TABLE IF NOT EXISTS scenarios_llm (
                scenario_id VARCHAR(255) PRIMARY KEY,
                parametres_json VARIANT,
                image_generee_url VARCHAR(1000),
                type_plan VARCHAR(100),
                nombre_elements INT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
            )
            """
        ]
        
        for query in queries:
            self.execute_query(query, fetch=False)
        
        print("✅ Schéma Snowflake créé avec succès!")
    
    def insert_plan_metadata(self, plan_data: Dict):
        """Insère les métadonnées d'un plan"""
        query = f"""
        INSERT INTO plans_metadata 
        (id, nom_fichier, date_upload, dimensions_width, dimensions_height, 
         nombre_pages, taille_fichier_kb, metadata_json)
        VALUES (
            '{plan_data['id']}',
            '{plan_data['nom_fichier']}',
            '{plan_data['date_upload']}',
            {plan_data['dimensions_width']},
            {plan_data['dimensions_height']},
            {plan_data['nombre_pages']},
            {plan_data['taille_fichier_kb']},
            PARSE_JSON('{json.dumps(plan_data.get('metadata_json', {}))}')
        )
        """
        return self.execute_query(query, fetch=False)
    
    def close(self):
        """Ferme la connexion"""
        if self.cursor:
            self.cursor.close()
        if self.conn:
            self.conn.close()
        print("✅ Connexion Snowflake fermée")

# Initialisation
snowflake_mgr = SnowflakeManager(config)
snowflake_connected = snowflake_mgr.connect()

if snowflake_connected:
    snowflake_mgr.create_schema()
else:
    print("💡 Le notebook fonctionnera en mode simulation (sans Snowflake réel)")


❌ Erreur connexion Snowflake: 290404 (08001): 404 Not Found: post yhurmit-us29400.snowflakecomputing.com.snowflakecomputing.com:443/session/v1/login-request
💡 Mode simulation activé (pas de connexion réelle)
💡 Le notebook fonctionnera en mode simulation (sans Snowflake réel)


# Section 3: Extraction et Preprocessing PDF

## ❓ Pourquoi convertir les PDF en images ?

### Raisons techniques principales :

1. **📸 Format natif des modèles de Deep Learning**
   - YOLO, et tous les modèles de computer vision, travaillent avec des **matrices de pixels** (images)
   - PyTorch/TensorFlow nécessitent des formats image (PNG, JPEG) pour les tensors
   - Les PDFs sont des formats **vectoriels/textuels complexes** non compatibles directement

2. **🔧 Preprocessing nécessaire**
   - Binarisation, dénoisage, correction d'orientation → nécessitent des images
   - OpenCV et scikit-image travaillent sur des arrays NumPy (représentation d'images)
   - Les opérations morphologiques (érosion, dilatation) s'appliquent aux pixels

3. **📊 Annotations YOLO**
   - Format YOLO: coordonnées normalisées basées sur les **dimensions de l'image** (width, height)
   - Les bounding boxes sont définies en pixels, pas en coordonnées vectorielles PDF
   - La segmentation nécessite des masques d'images (matrices binaires)

4. **⚡ Performance et standardisation**
   - Images = format universel et standardisé pour computer vision
   - Plus rapide à traiter qu'un parsing complexe de PDF
   - Compatible avec toutes les bibliothèques de vision par ordinateur

5. **🔄 Pipeline cohérent**
   - Toutes les étapes (augmentation, labellisation, entraînement) utilisent des images
   - Facilite le workflow: PDF → Image → Traitement → Export vectoriel

### ⚠️ Note importante :
- La conversion PDF → Image est une **étape intermédiaire nécessaire**
- On **reconvertit ensuite en format vectoriel** (DXF, IFC) à la fin du pipeline
- On ne perd pas la qualité: conversion haute résolution (300 DPI minimum)

---

## 3.1 Extraction des pages PDF en images


In [7]:
# ============================================================================
# EXTRACTION PDF → IMAGES
# ============================================================================

class PDFExtractor:
    """Extracteur de pages PDF vers images"""
    
    def __init__(self, config):
        self.config = config
    
    def extract_pages_pymupdf(self, pdf_path: str) -> List[Image.Image]:
        """
        Extrait les pages d'un PDF en images haute qualité avec PyMuPDF
        
        Args:
            pdf_path: Chemin vers le fichier PDF
            
        Returns:
            Liste d'images PIL
        """
        images = []
        try:
            doc = fitz.open(pdf_path)
            print(f"📄 PDF ouvert: {len(doc)} pages")
            
            for page_num in range(len(doc)):
                page = doc[page_num]
                
                # Matrice de transformation pour haute résolution
                mat = fitz.Matrix(self.config.PDF_SCALE, self.config.PDF_SCALE)
                pix = page.get_pixmap(matrix=mat)
                
                # Conversion en PIL Image
                img_data = pix.tobytes("png")
                img = Image.open(BytesIO(img_data))
                images.append(img)
                
                print(f"  ✓ Page {page_num + 1} extraite: {img.size}")
            
            doc.close()
            return images
            
        except Exception as e:
            print(f"❌ Erreur extraction PyMuPDF: {e}")
            return []
    
    def extract_pages_pdf2image(self, pdf_path: str) -> List[Image.Image]:
        """
        Alternative: Extraction avec pdf2image (nécessite poppler)
        
        Args:
            pdf_path: Chemin vers le fichier PDF
            
        Returns:
            Liste d'images PIL
        """
        try:
            images = convert_from_path(
                pdf_path,
                dpi=self.config.PDF_DPI,
                fmt='png'
            )
            print(f"📄 {len(images)} pages extraites avec pdf2image")
            return images
        except Exception as e:
            print(f"❌ Erreur extraction pdf2image: {e}")
            return []
    
    def save_extracted_images(self, images: List[Image.Image], 
                             pdf_name: str, output_dir: Path) -> List[str]:
        """
        Sauvegarde les images extraites
        
        Args:
            images: Liste d'images PIL
            pdf_name: Nom du PDF (sans extension)
            output_dir: Répertoire de sortie
            
        Returns:
            Liste des chemins des images sauvegardées
        """
        saved_paths = []
        output_dir.mkdir(parents=True, exist_ok=True)
        
        for idx, img in enumerate(images):
            img_path = output_dir / f"{pdf_name}_page_{idx+1:03d}.png"
            img.save(img_path, "PNG", quality=95)
            saved_paths.append(str(img_path))
            print(f"  💾 Image sauvegardée: {img_path.name}")
        
        return saved_paths
    
    def get_pdf_metadata(self, pdf_path: str) -> Dict:
        """Extrait les métadonnées d'un PDF"""
        try:
            doc = fitz.open(pdf_path)
            metadata = doc.metadata
            doc.close()
            
            file_size = os.path.getsize(pdf_path) / 1024  # KB
            
            return {
                'nombre_pages': len(doc),
                'taille_fichier_kb': file_size,
                'titre': metadata.get('title', ''),
                'auteur': metadata.get('author', ''),
                'sujet': metadata.get('subject', ''),
                'date_creation': metadata.get('creationDate', '')
            }
        except Exception as e:
            print(f"❌ Erreur métadonnées: {e}")
            return {}

# Test d'extraction
extractor = PDFExtractor(config)

# Exemple d'utilisation (remplacer par vos PDFs)
print("📝 Exemple d'extraction PDF:")
print("Pour tester, placez vos fichiers PDF dans le dossier data/pdfs/")

# Fonction helper pour traiter un PDF
def process_pdf(pdf_path: str) -> Dict:
    """Traite un PDF complet"""
    pdf_name = Path(pdf_path).stem
    
    # Extraction
    images = extractor.extract_pages_pymupdf(pdf_path)
    if not images:
        images = extractor.extract_pages_pdf2image(pdf_path)
    
    if not images:
        return None
    
    # Sauvegarde
    saved_paths = extractor.save_extracted_images(
        images, pdf_name, config.IMAGES_DIR
    )
    
    # Métadonnées
    metadata = extractor.get_pdf_metadata(pdf_path)
    first_img = images[0]
    
    # Stockage Snowflake
    plan_data = {
        'id': f"plan_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        'nom_fichier': Path(pdf_path).name,
        'date_upload': datetime.now().isoformat(),
        'dimensions_width': first_img.width,
        'dimensions_height': first_img.height,
        'nombre_pages': len(images),
        'taille_fichier_kb': metadata.get('taille_fichier_kb', 0),
        'metadata_json': metadata
    }
    
    if snowflake_connected:
        snowflake_mgr.insert_plan_metadata(plan_data)
    
    return {
        'plan_data': plan_data,
        'images': images,
        'saved_paths': saved_paths
    }

print("✅ Extracteur PDF initialisé!")


📝 Exemple d'extraction PDF:
Pour tester, placez vos fichiers PDF dans le dossier data/pdfs/
✅ Extracteur PDF initialisé!


In [8]:
# ============================================================================
# PREPROCESSING IMAGES
# ============================================================================

class ImagePreprocessor:
    """Préprocesseur d'images pour plans architecturaux"""
    
    def __init__(self, config):
        self.config = config
    
    def detect_orientation(self, image: np.ndarray) -> int:
        """
        Détecte l'orientation de l'image (0, 90, 180, 270 degrés)
        Utilise la détection de texte/contours pour déterminer l'orientation
        """
        # Conversion en niveaux de gris
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) if len(image.shape) == 3 else image
        
        # Détection de contours
        edges = cv2.Canny(gray, 50, 150)
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, 
                                minLineLength=50, maxLineGap=10)
        
        if lines is not None and len(lines) > 0:
            angles = []
            for line in lines:
                x1, y1, x2, y2 = line[0]
                angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
                angles.append(angle)
            
            # Angle dominant
            angle_hist, _ = np.histogram(angles, bins=36, range=(-90, 90))
            dominant_angle = np.argmax(angle_hist) * 5 - 90
            
            # Normalisation à 0, 90, 180, 270
            if abs(dominant_angle) < 45:
                return 0
            elif 45 <= dominant_angle < 135:
                return 90
            elif -135 <= dominant_angle < -45:
                return -90
            else:
                return 180
        
        return 0
    
    def correct_orientation(self, image: Image.Image) -> Image.Image:
        """Corrige l'orientation de l'image"""
        img_array = np.array(image)
        angle = self.detect_orientation(img_array)
        
        if angle != 0:
            # Rotation
            if angle == 90:
                return image.rotate(-90, expand=True)
            elif angle == -90 or angle == 270:
                return image.rotate(90, expand=True)
            elif angle == 180:
                return image.rotate(180, expand=True)
        
        return image
    
    def binarize_otsu(self, image: np.ndarray) -> np.ndarray:
        """Binarisation avec méthode d'Otsu"""
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image.copy()
        
        # Binarisation Otsu
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return binary
    
    def binarize_adaptive(self, image: np.ndarray, block_size: int = 11, 
                         C: int = 2) -> np.ndarray:
        """Binarisation adaptative"""
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image.copy()
        
        binary = cv2.adaptiveThreshold(
            gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
            cv2.THRESH_BINARY, block_size, C
        )
        return binary
    
    def denoise(self, image: np.ndarray, method: str = 'morphological') -> np.ndarray:
        """
        Dénoisage de l'image
        
        Args:
            image: Image binarisée
            method: 'morphological' ou 'gaussian'
        """
        if method == 'morphological':
            # Opérations morphologiques
            kernel = np.ones((3, 3), np.uint8)
            denoised = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
            denoised = cv2.morphologyEx(denoised, cv2.MORPH_OPEN, kernel)
            return denoised
        
        elif method == 'gaussian':
            # Filtre gaussien (pour images non binarisées)
            return cv2.GaussianBlur(image, (5, 5), 0)
        
        return image
    
    def normalize_size(self, image: Image.Image, target_size: Tuple[int, int] = None) -> Image.Image:
        """Normalise la taille de l'image"""
        if target_size is None:
            target_size = self.config.IMAGE_SIZE
        
        # Resize en conservant le ratio d'aspect
        image.thumbnail(target_size, Image.Resampling.LANCZOS)
        
        # Création d'une nouvelle image avec la taille cible
        new_img = Image.new('RGB', target_size, color='white')
        new_img.paste(image, ((target_size[0] - image.width) // 2,
                             (target_size[1] - image.height) // 2))
        
        return new_img
    
    def correct_perspective(self, image: np.ndarray) -> np.ndarray:
        """Corrige la perspective si le plan est incliné"""
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) if len(image.shape) == 3 else image
        
        # Détection de contours
        edges = cv2.Canny(gray, 50, 150)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(contours) > 0:
            # Plus grand contour (probablement le plan)
            largest_contour = max(contours, key=cv2.contourArea)
            
            # Approximation polygonale
            epsilon = 0.02 * cv2.arcLength(largest_contour, True)
            approx = cv2.approxPolyDP(largest_contour, epsilon, True)
            
            if len(approx) == 4:
                # Transformation de perspective
                pts1 = np.float32(approx.reshape(4, 2))
                pts2 = np.float32([[0, 0], [image.shape[1], 0],
                                  [image.shape[1], image.shape[0]], [0, image.shape[0]]])
                
                M = cv2.getPerspectiveTransform(pts1, pts2)
                corrected = cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]))
                return corrected
        
        return image
    
    def preprocess_pipeline(self, image: Image.Image, 
                           steps: List[str] = None) -> Image.Image:
        """
        Pipeline complet de preprocessing
        
        Args:
            image: Image PIL à traiter
            steps: Liste des étapes ['orientation', 'binarize', 'denoise', 'normalize']
        """
        if steps is None:
            steps = ['orientation', 'binarize', 'denoise', 'normalize']
        
        img_array = np.array(image)
        
        # Correction orientation
        if 'orientation' in steps:
            image = self.correct_orientation(image)
            img_array = np.array(image)
        
        # Binarisation
        if 'binarize' in steps:
            img_array = self.binarize_otsu(img_array)
            # Conversion en RGB pour PIL
            img_array = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
        
        # Dénoisage
        if 'denoise' in steps:
            if len(img_array.shape) == 3:
                gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
            else:
                gray = img_array
            gray = self.denoise(gray, method='morphological')
            img_array = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
        
        # Normalisation taille
        if 'normalize' in steps:
            image = Image.fromarray(img_array)
            image = self.normalize_size(image)
        else:
            image = Image.fromarray(img_array)
        
        return image

# Initialisation
preprocessor = ImagePreprocessor(config)

# Fonction helper
def preprocess_image(image_path: str, save_path: str = None) -> Image.Image:
    """Préprocesse une image depuis un fichier"""
    image = Image.open(image_path)
    processed = preprocessor.preprocess_pipeline(image)
    
    if save_path:
        processed.save(save_path)
        print(f"💾 Image préprocessée sauvegardée: {save_path}")
    
    return processed

print("✅ Preprocesseur initialisé!")


✅ Preprocesseur initialisé!


In [ ]:
# ============================================================================
# GÉNÉRATION DE DONNÉES AVEC LLM
# ============================================================================

class LLMDataGenerator:
    """Générateur de données synthétiques avec LLM"""
    
    def __init__(self, config):
        self.config = config
        self.client = None
        
        if ANTHROPIC_AVAILABLE:
            try:
                self.client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY", ""))
                print("✅ Client Anthropic initialisé")
            except:
                print("⚠️ Anthropic non configuré, utilisation modèle local")
    
    def generate_scenarios_prompt(self, num_plans: int = 20) -> str:
        """
        Génère le prompt pour créer des scénarios de plans
        
        Args:
            num_plans: Nombre de plans réels analysés
        """
        prompt = f"""
Tu es un expert en architecture. Analyse {num_plans} plans PDF d'architecte et génère {self.config.NUM_SCENARIOS} scénarios JSON différents et variés.

Chaque scénario doit représenter un plan architectural réaliste avec les caractéristiques suivantes:

STRUCTURE JSON REQUISE:
{{
    "scenario_id": "unique_id",
    "type_plan": "appartement" | "maison" | "bureau" | "commercial" | "industriel",
    "nombre_pieces": integer (2-15),
    "superficie_totale_m2": float,
    "dimensions": {{
        "longueur_m": float,
        "largeur_m": float,
        "hauteur_m": float (optionnel)
    }},
    "elements": [
        {{
            "type": "porte" | "fenetre" | "mur" | "escalier" | "colonne" | "sanitaire" | "cuisine",
            "position": [x_m, y_m],  # Coordonnées en mètres
            "dimensions": {{
                "longueur_m": float,
                "largeur_m": float
            }},
            "orientation": 0 | 90 | 180 | 270,
            "style": "moderne" | "classique" | "contemporain"
        }},
        ...
    ],
    "pieces": [
        {{
            "nom": "salon" | "chambre" | "cuisine" | "salle_de_bain" | etc.,
            "superficie_m2": float,
            "position": [x_m, y_m],
            "dimensions": {{"longueur_m": float, "largeur_m": float}},
            "elements": ["porte_1", "fenetre_1", ...]  # IDs des éléments
        }},
        ...
    ],
    "style_architectural": "moderne" | "classique" | "contemporain" | "minimaliste",
    "echelle": "1:100" | "1:50" | "1:200",
    "orientation_batiment": "nord" | "sud" | "est" | "ouest"
}}

CONTRAINTES:
- Les coordonnées doivent être cohérentes (pas d'éléments qui se chevauchent)
- Les dimensions doivent être réalistes (portes: 0.8-1.2m, fenêtres: 0.6-2m)
- Varier les types de plans (appartements, maisons, bureaux)
- Inclure différents styles architecturaux
- Assurer la cohérence spatiale

Génère UNIQUEMENT un tableau JSON valide avec {self.config.NUM_SCENARIOS} objets, sans texte supplémentaire.
"""
        return prompt
    
    def generate_scenarios_anthropic(self) -> List[Dict]:
        """Génère des scénarios avec Anthropic Claude"""
        if not self.client:
            print("⚠️ Anthropic non disponible, génération de scénarios mock")
            return self._generate_mock_scenarios()
        
        try:
            prompt = self.generate_scenarios_prompt()
            
            # Déterminer le modèle Anthropic (par défaut claude-3-5-sonnet-20241022)
            model = self.config.LLM_MODEL if hasattr(self.config, 'LLM_MODEL') and 'claude' in self.config.LLM_MODEL.lower() else "claude-3-5-sonnet-20241022"
            
            response = self.client.messages.create(
                model=model,
                max_tokens=4096,
                temperature=self.config.LLM_TEMPERATURE,
                system="Tu es un expert en architecture et génération de données structurées. Tu génères toujours du JSON valide.",
                messages=[
                    {"role": "user", "content": prompt + "\n\nIMPORTANT: Réponds UNIQUEMENT avec un tableau JSON valide, sans texte supplémentaire."}
                ]
            )
            
            content = response.content[0].text
            
            # Parsing JSON
            try:
                # Tentative d'extraction JSON depuis le texte
                import re
                # Chercher un tableau JSON
                json_match = re.search(r'\[.*\]', content, re.DOTALL)
                if json_match:
                    data = json.loads(json_match.group())
                    if isinstance(data, list):
                        scenarios = data
                    else:
                        scenarios = [data]
                else:
                    # Essayer de parser directement
                    data = json.loads(content)
                    if isinstance(data, dict) and "scenarios" in data:
                        scenarios = data["scenarios"]
                    elif isinstance(data, list):
                        scenarios = data
                    else:
                        scenarios = [data]
                
                print(f"✅ {len(scenarios)} scénarios générés avec Anthropic")
                return scenarios[:self.config.NUM_SCENARIOS]
                
            except json.JSONDecodeError:
                print("⚠️ Erreur parsing JSON, extraction manuelle...")
                # Tentative d'extraction JSON depuis le texte
                import re
                json_match = re.search(r'\[.*\]', content, re.DOTALL)
                if json_match:
                    scenarios = json.loads(json_match.group())
                    return scenarios[:self.config.NUM_SCENARIOS]
                return self._generate_mock_scenarios()
                
        except Exception as e:
            print(f"❌ Erreur génération Anthropic: {e}")
            return self._generate_mock_scenarios()
    
    def generate_scenarios_openai(self) -> List[Dict]:
        """Alias pour compatibilité - utilise Anthropic"""
        return self.generate_scenarios_anthropic()
    
    def _generate_mock_scenarios(self, num: int = None) -> List[Dict]:
        """Génère des scénarios mock pour tests"""
        if num is None:
            num = self.config.NUM_SCENARIOS
        
        import random
        scenarios = []
        
        types_plans = ["appartement", "maison", "bureau", "commercial"]
        types_elements = ["porte", "fenetre", "mur", "escalier", "colonne", "sanitaire"]
        styles = ["moderne", "classique", "contemporain"]
        
        for i in range(num):
            type_plan = random.choice(types_plans)
            nb_pieces = random.randint(2, 10)
            
            scenario = {
                "scenario_id": f"scenario_{i+1:03d}",
                "type_plan": type_plan,
                "nombre_pieces": nb_pieces,
                "superficie_totale_m2": random.uniform(30, 200),
                "dimensions": {
                    "longueur_m": random.uniform(5, 20),
                    "largeur_m": random.uniform(4, 15)
                },
                "elements": [],
                "pieces": [],
                "style_architectural": random.choice(styles),
                "echelle": random.choice(["1:100", "1:50", "1:200"]),
                "orientation_batiment": random.choice(["nord", "sud", "est", "ouest"])
            }
            
            # Génération éléments
            nb_elements = random.randint(5, 20)
            for j in range(nb_elements):
                elem_type = random.choice(types_elements)
                scenario["elements"].append({
                    "type": elem_type,
                    "position": [
                        random.uniform(0, scenario["dimensions"]["longueur_m"]),
                        random.uniform(0, scenario["dimensions"]["largeur_m"])
                    ],
                    "dimensions": {
                        "longueur_m": random.uniform(0.5, 3.0),
                        "largeur_m": random.uniform(0.5, 2.0)
                    },
                    "orientation": random.choice([0, 90, 180, 270]),
                    "style": random.choice(styles)
                })
            
            scenarios.append(scenario)
        
        print(f"✅ {len(scenarios)} scénarios mock générés")
        return scenarios
    
    def save_scenarios_snowflake(self, scenarios: List[Dict]):
        """Sauvegarde les scénarios dans Snowflake"""
        if not snowflake_connected:
            print("⚠️ Snowflake non disponible, sauvegarde locale")
            # Sauvegarde locale
            scenarios_path = config.DATA_DIR / "scenarios_llm.json"
            with open(scenarios_path, 'w', encoding='utf-8') as f:
                json.dump(scenarios, f, indent=2, ensure_ascii=False)
            print(f"💾 Scénarios sauvegardés: {scenarios_path}")
            return
        
        for scenario in scenarios:
            query = f"""
            INSERT INTO scenarios_llm 
            (scenario_id, parametres_json, type_plan, nombre_elements)
            VALUES (
                '{scenario['scenario_id']}',
                PARSE_JSON('{json.dumps(scenario)}'),
                '{scenario['type_plan']}',
                {len(scenario.get('elements', []))}
            )
            """
            snowflake_mgr.execute_query(query, fetch=False)
        
        print(f"✅ {len(scenarios)} scénarios sauvegardés dans Snowflake")

# Initialisation
llm_generator = LLMDataGenerator(config)

# Génération des scénarios
print("🔄 Génération de scénarios avec LLM...")
scenarios = llm_generator.generate_scenarios_openai()

# Sauvegarde
llm_generator.save_scenarios_snowflake(scenarios)

print(f"\n📊 Exemple de scénario généré:")
if scenarios:
    print(json.dumps(scenarios[0], indent=2, ensure_ascii=False))


✅ Client OpenAI initialisé
🔄 Génération de scénarios avec LLM...
❌ Erreur génération OpenAI: Error code: 401 - {'error': {'message': "You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
✅ 50 scénarios mock générés
⚠️ Snowflake non disponible, sauvegarde locale
💾 Scénarios sauvegardés: data\scenarios_llm.json

📊 Exemple de scénario généré:
{
  "scenario_id": "scenario_001",
  "type_plan": "appartement",
  "nombre_pieces": 9,
  "superficie_totale_m2": 97.55800549117619,
  "dimensions": {
    "longueur_m": 12.597383246960582,
    "largeur_m": 12.277849324923924
  },
  "elements": [
    {
      "type": "mur",
      "pos

: 

In [ ]:
# ============================================================================
# GÉNÉRATION D'IMAGES SYNTHÉTIQUES
# ============================================================================

class SyntheticImageGenerator:
    """Générateur d'images synthétiques de plans avec IA générative"""
    
    def __init__(self, config):
        self.config = config
        self.diffusion_pipeline = None
        
        # Tentative de chargement Stable Diffusion
        try:
            from diffusers import StableDiffusionPipeline
            import torch
            
            if torch.cuda.is_available():
                self.diffusion_pipeline = StableDiffusionPipeline.from_pretrained(
                    "runwayml/stable-diffusion-v1-5",
                    torch_dtype=torch.float16
                ).to("cuda")
            else:
                self.diffusion_pipeline = StableDiffusionPipeline.from_pretrained(
                    "runwayml/stable-diffusion-v1-5"
                )
            print("✅ Pipeline Stable Diffusion chargé")
        except Exception as e:
            print(f"⚠️ Stable Diffusion non disponible: {e}")
            print("💡 Utilisation de génération procédurale")
    
    def scenario_to_prompt(self, scenario: Dict) -> str:
        """Convertit un scénario JSON en prompt pour génération d'image"""
        type_plan = scenario.get('type_plan', 'appartement')
        style = scenario.get('style_architectural', 'moderne')
        nb_pieces = scenario.get('nombre_pieces', 3)
        
        elements_desc = []
        for elem in scenario.get('elements', [])[:10]:  # Limiter pour le prompt
            elements_desc.append(elem['type'])
        
        prompt = f"""
Architectural floor plan, {type_plan}, {style} style, {nb_pieces} rooms,
elements: {', '.join(set(elements_desc))},
technical drawing, black and white, clean lines, 2D top view,
high quality, detailed, professional architecture blueprint
"""
        return prompt.strip()
    
    def generate_image_stable_diffusion(self, prompt: str, 
                                       num_inference_steps: int = 50) -> Image.Image:
        """Génère une image avec Stable Diffusion"""
        if not self.diffusion_pipeline:
            return self.generate_image_procedural(prompt)
        
        try:
            negative_prompt = "color, 3D, perspective, photo, realistic, messy, low quality"
            
            image = self.diffusion_pipeline(
                prompt=prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=num_inference_steps,
                guidance_scale=7.5,
                height=512,
                width=512
            ).images[0]
            
            return image
        except Exception as e:
            print(f"❌ Erreur génération Stable Diffusion: {e}")
            return self.generate_image_procedural(prompt)
    
    def generate_image_procedural(self, scenario: Dict) -> Image.Image:
        """
        Génère une image procédurale basée sur le scénario JSON
        Alternative si Stable Diffusion n'est pas disponible
        """
        width, height = 1024, 1024
        img = Image.new('RGB', (width, height), color='white')
        draw = ImageDraw.Draw(img)
        
        # Dimensions du plan
        plan_w = scenario['dimensions']['longueur_m']
        plan_h = scenario['dimensions']['largeur_m']
        
        # Échelle pour le dessin
        scale_x = (width - 100) / plan_w
        scale_y = (height - 100) / plan_h
        scale = min(scale_x, scale_y)
        
        offset_x = (width - plan_w * scale) / 2
        offset_y = (height - plan_h * scale) / 2
        
        # Dessin des murs (contour)
        wall_thickness = 20
        draw.rectangle(
            [offset_x, offset_y, 
             offset_x + plan_w * scale, offset_y + plan_h * scale],
            outline='black', width=wall_thickness
        )
        
        # Dessin des éléments
        for elem in scenario.get('elements', []):
            elem_type = elem['type']
            pos = elem['position']
            dims = elem['dimensions']
            
            x = offset_x + pos[0] * scale
            y = offset_y + pos[1] * scale
            w = dims['longueur_m'] * scale
            h = dims['largeur_m'] * scale
            
            if elem_type == 'porte':
                # Porte: rectangle avec arc
                draw.rectangle([x, y, x + w, y + h], outline='black', width=3)
                # Arc de cercle pour l'ouverture
                draw.arc([x - w/2, y, x + w/2, y + h], start=0, end=180, 
                        fill='black', width=2)
            elif elem_type == 'fenetre':
                # Fenêtre: rectangle double ligne
                draw.rectangle([x, y, x + w, y + h], outline='black', width=3)
                draw.rectangle([x + 5, y + 5, x + w - 5, y + h - 5], 
                             outline='black', width=2)
            elif elem_type == 'mur':
                # Mur: ligne épaisse
                draw.rectangle([x, y, x + w, y + h], fill='black')
            elif elem_type == 'escalier':
                # Escalier: rectangles en escalier
                for i in range(5):
                    draw.rectangle([x + i*10, y + i*10, x + w, y + h/5 + i*10],
                                 outline='black', width=2)
            else:
                # Autres éléments: rectangle simple
                draw.rectangle([x, y, x + w, y + h], outline='black', width=2)
        
        return img
    
    def generate_images_from_scenarios(self, scenarios: List[Dict], 
                                      output_dir: Path) -> List[str]:
        """Génère des images pour tous les scénarios"""
        output_dir.mkdir(parents=True, exist_ok=True)
        generated_paths = []
        
        for scenario in scenarios:
            scenario_id = scenario['scenario_id']
            
            # Génération
            if self.diffusion_pipeline:
                prompt = self.scenario_to_prompt(scenario)
                image = self.generate_image_stable_diffusion(prompt)
            else:
                image = self.generate_image_procedural(scenario)
            
            # Sauvegarde
            img_path = output_dir / f"{scenario_id}.png"
            image.save(img_path, "PNG")
            generated_paths.append(str(img_path))
            
            print(f"  ✅ Image générée: {scenario_id}")
        
        print(f"✅ {len(generated_paths)} images synthétiques générées")
        return generated_paths

# Initialisation
image_generator = SyntheticImageGenerator(config)

# Génération d'images (exemple avec les 5 premiers scénarios)
if scenarios:
    print("🔄 Génération d'images synthétiques...")
    synthetic_images_dir = config.DATA_DIR / "synthetic_images"
    generated_paths = image_generator.generate_images_from_scenarios(
        scenarios[:5],  # Limiter pour la démo
        synthetic_images_dir
    )
    
    # Affichage d'un exemple
    if generated_paths:
        example_img = Image.open(generated_paths[0])
        plt.figure(figsize=(10, 10))
        plt.imshow(example_img)
        plt.title(f"Exemple d'image synthétique: {Path(generated_paths[0]).stem}")
        plt.axis('off')
        plt.show()


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


diffusion_pytorch_model.safetensors:  69%|######8   | 2.36G/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# ============================================================================
# AUGMENTATION DE DONNÉES
# ============================================================================

class DataAugmenter:
    """Augmente les données d'entraînement"""
    
    def __init__(self, config):
        self.config = config
    
    def rotate_image(self, image: Image.Image, angle: int) -> Image.Image:
        """Rotation de l'image"""
        return image.rotate(angle, expand=True, fillcolor='white')
    
    def flip_image(self, image: Image.Image, direction: str = 'horizontal') -> Image.Image:
        """Retournement horizontal ou vertical"""
        if direction == 'horizontal':
            return image.transpose(Image.FLIP_LEFT_RIGHT)
        elif direction == 'vertical':
            return image.transpose(Image.FLIP_TOP_BOTTOM)
        return image
    
    def scale_image(self, image: Image.Image, scale_factor: float) -> Image.Image:
        """Redimensionnement avec facteur d'échelle"""
        new_size = (int(image.width * scale_factor), int(image.height * scale_factor))
        return image.resize(new_size, Image.Resampling.LANCZOS)
    
    def add_noise(self, image: np.ndarray, noise_level: float = 0.1) -> np.ndarray:
        """Ajoute du bruit gaussien"""
        noise = np.random.normal(0, noise_level * 255, image.shape).astype(np.uint8)
        noisy = cv2.add(image.astype(np.uint8), noise)
        return np.clip(noisy, 0, 255)
    
    def adjust_brightness(self, image: Image.Image, factor: float) -> Image.Image:
        """Ajuste la luminosité"""
        from PIL import ImageEnhance
        enhancer = ImageEnhance.Brightness(image)
        return enhancer.enhance(factor)
    
    def augment_image(self, image: Image.Image, 
                     augmentations: List[str] = None) -> List[Image.Image]:
        """
        Applique plusieurs augmentations à une image
        
        Args:
            image: Image à augmenter
            augmentations: Liste des types d'augmentation à appliquer
            
        Returns:
            Liste d'images augmentées
        """
        if augmentations is None:
            augmentations = ['rotate', 'flip', 'scale', 'noise', 'brightness']
        
        augmented_images = []
        img_array = np.array(image)
        
        # Rotations
        if 'rotate' in augmentations:
            for angle in self.config.AUGMENTATION_ROTATIONS:
                rotated = self.rotate_image(image, angle)
                augmented_images.append(rotated)
        
        # Retournements
        if 'flip' in augmentations and self.config.AUGMENTATION_FLIP:
            flipped_h = self.flip_image(image, 'horizontal')
            flipped_v = self.flip_image(image, 'vertical')
            augmented_images.extend([flipped_h, flipped_v])
        
        # Redimensionnements
        if 'scale' in augmentations:
            for scale in self.config.AUGMENTATION_SCALE:
                if scale != 1.0:
                    scaled = self.scale_image(image, scale)
                    # Recadrer à la taille originale
                    scaled = scaled.crop((0, 0, image.width, image.height))
                    augmented_images.append(scaled)
        
        # Bruit
        if 'noise' in augmentations and self.config.AUGMENTATION_NOISE:
            noisy = self.add_noise(img_array, noise_level=0.05)
            augmented_images.append(Image.fromarray(noisy))
        
        # Luminosité
        if 'brightness' in augmentations:
            bright = self.adjust_brightness(image, 1.2)
            dark = self.adjust_brightness(image, 0.8)
            augmented_images.extend([bright, dark])
        
        return augmented_images
    
    def augment_dataset(self, image_paths: List[str], 
                       output_dir: Path, prefix: str = "aug") -> List[str]:
        """
        Augmente un dataset complet
        
        Args:
            image_paths: Liste des chemins d'images
            output_dir: Répertoire de sortie
            prefix: Préfixe pour les images augmentées
            
        Returns:
            Liste des chemins des images augmentées
        """
        output_dir.mkdir(parents=True, exist_ok=True)
        augmented_paths = []
        
        for img_path in image_paths:
            image = Image.open(img_path)
            base_name = Path(img_path).stem
            
            # Augmentations
            augmented = self.augment_image(image)
            
            # Sauvegarde
            for idx, aug_img in enumerate(augmented):
                aug_path = output_dir / f"{prefix}_{base_name}_{idx:03d}.png"
                aug_img.save(aug_path, "PNG")
                augmented_paths.append(str(aug_path))
            
            print(f"  ✅ {len(augmented)} augmentations pour {base_name}")
        
        print(f"✅ {len(augmented_paths)} images augmentées générées")
        return augmented_paths

# Initialisation
augmenter = DataAugmenter(config)

# Exemple d'augmentation (si vous avez des images)
print("📝 Pour augmenter vos images:")
print("augmented_paths = augmenter.augment_dataset(image_paths, output_dir)")


In [ ]:
# ============================================================================
# LABELLISATION AUTOMATIQUE AVEC YOLO
# ============================================================================
#
# ⚠️ IMPORTANT: YOLO pré-entraîné détecte des objets génériques (COCO dataset)
#    - Classes détectées: person, car, bicycle, chair, etc. (80 classes COCO)
#    - PAS d'éléments architecturaux spécifiques (porte, fenêtre, mur, etc.)
#
# Pour détecter des éléments architecturaux:
#    1. Fine-tuner YOLO sur un dataset de plans architecturaux (Section 6)
#    2. Utiliser un modèle pré-entraîné spécialisé en architecture
#    3. Utiliser un modèle de segmentation sémantique spécialisé
#
# Le code ci-dessous utilise les noms de classes réels de YOLO (result.names)
# au lieu de config.CLASSES qui est pour un modèle fine-tuné.
# ============================================================================

class YOLOLabeler:
    """Labellisateur automatique avec YOLO pré-entraîné"""
    
    def __init__(self, config):
        self.config = config
        self.model_detection = None
        self.model_segmentation = None
        self.model_keypoints = None
        
        self._load_models()
    
    def _load_models(self):
        """Charge les modèles YOLO pré-entraînés"""
        try:
            # Modèle détection
            self.model_detection = YOLO(self.config.YOLO_MODEL_DETECTION)
            print(f"✅ Modèle détection chargé: {self.config.YOLO_MODEL_DETECTION}")
            
            # Modèle segmentation
            self.model_segmentation = YOLO(self.config.YOLO_MODEL_SEGMENTATION)
            print(f"✅ Modèle segmentation chargé: {self.config.YOLO_MODEL_SEGMENTATION}")
            
            # Modèle keypoints
            self.model_keypoints = YOLO(self.config.YOLO_MODEL_KEYPOINTS)
            print(f"✅ Modèle keypoints chargé: {self.config.YOLO_MODEL_KEYPOINTS}")
            
        except Exception as e:
            print(f"⚠️ Erreur chargement modèles YOLO: {e}")
            print("💡 Téléchargez les modèles depuis: https://github.com/ultralytics/ultralytics")
    
    def predict_detection(self, image_path: str) -> List[Dict]:
        """
        Prédit les détections sur une image
        
        Returns:
            Liste de détections avec bbox, confidence, classe
        """
        if not self.model_detection:
            return []
        
        try:
            results = self.model_detection.predict(
                image_path,
                imgsz=self.config.YOLO_IMG_SIZE,
                conf=self.config.YOLO_CONF_THRESHOLD,
                iou=self.config.YOLO_IOU_THRESHOLD,
                verbose=False
            )
            
            detections = []
            for result in results:
                boxes = result.boxes
                # Utiliser les noms de classes réels de YOLO (COCO dataset)
                # YOLO détecte des objets génériques, pas des éléments architecturaux spécifiques
                # Pour détecter des éléments architecturaux, il faudrait un modèle fine-tuné
                for box in boxes:
                    # Coordonnées normalisées YOLO (x_center, y_center, width, height)
                    x_center, y_center, width, height = box.xywhn[0].cpu().numpy()
                    conf = float(box.conf[0].cpu().numpy())
                    cls = int(box.cls[0].cpu().numpy())
                    
                    # Utiliser les noms de classes réels de YOLO (result.names)
                    # Au lieu de config.CLASSES qui est pour un modèle fine-tuné
                    class_name = result.names[cls] if hasattr(result, 'names') and cls in result.names else f"class_{cls}"
                    
                    detections.append({
                        'class': cls,
                        'class_name': class_name,
                        'confidence': conf,
                        'bbox': {
                            'x_center': float(x_center),
                            'y_center': float(y_center),
                            'width': float(width),
                            'height': float(height)
                        }
                    })
            
            return detections
            
        except Exception as e:
            print(f"❌ Erreur prédiction détection: {e}")
            return []
    
    def predict_segmentation(self, image_path: str) -> List[Dict]:
        """Prédit la segmentation sémantique"""
        if not self.model_segmentation:
            return []
        
        try:
            results = self.model_segmentation.predict(
                image_path,
                imgsz=self.config.YOLO_IMG_SIZE,
                conf=self.config.YOLO_CONF_THRESHOLD,
                verbose=False
            )
            
            segmentations = []
            for result in results:
                if result.masks is not None:
                    for mask, box in zip(result.masks, result.boxes):
                        # Points de segmentation
                        seg_points = mask.xy[0].cpu().numpy()
                        cls = int(box.cls[0].cpu().numpy())
                        conf = float(box.conf[0].cpu().numpy())
                        
                        # Utiliser les noms de classes réels de YOLO
                        class_name = result.names[cls] if hasattr(result, 'names') and cls in result.names else f"class_{cls}"
                        segmentations.append({
                            'class': cls,
                            'class_name': class_name,
                            'confidence': conf,
                            'segmentation_points': seg_points.tolist()
                        })
            
            return segmentations
            
        except Exception as e:
            print(f"❌ Erreur prédiction segmentation: {e}")
            return []
    
    def predict_keypoints(self, image_path: str) -> List[Dict]:
        """Prédit les keypoints"""
        if not self.model_keypoints:
            return []
        
        try:
            results = self.model_keypoints.predict(
                image_path,
                imgsz=self.config.YOLO_IMG_SIZE,
                conf=self.config.YOLO_CONF_THRESHOLD,
                verbose=False
            )
            
            keypoints_list = []
            for result in results:
                if result.keypoints is not None:
                    for kpts, box in zip(result.keypoints, result.boxes):
                        kpts_array = kpts.xy[0].cpu().numpy()
                        cls = int(box.cls[0].cpu().numpy())
                        conf = float(box.conf[0].cpu().numpy())
                        
                        keypoints_list.append({
                            'class': cls,
                            'confidence': conf,
                            'keypoints': kpts_array.tolist()
                        })
            
            return keypoints_list
            
        except Exception as e:
            print(f"❌ Erreur prédiction keypoints: {e}")
            return []
    
    def export_yolo_format(self, detections: List[Dict], 
                          image_path: str, output_dir: Path) -> str:
        """
        Exporte les annotations au format YOLO
        
        Format YOLO: class_id x_center y_center width height (toutes valeurs normalisées 0-1)
        """
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # Fichier .txt avec le même nom que l'image
        txt_path = output_dir / f"{Path(image_path).stem}.txt"
        
        with open(txt_path, 'w') as f:
            for det in detections:
                bbox = det['bbox']
                line = f"{det['class']} {bbox['x_center']} {bbox['y_center']} {bbox['width']} {bbox['height']}\n"
                f.write(line)
        
        return str(txt_path)
    
    def label_image(self, image_path: str, save_annotations: bool = True) -> Dict:
        """
        Labellise complètement une image (détection + segmentation + keypoints)
        
        Returns:
            Dictionnaire avec toutes les annotations
        """
        detections = self.predict_detection(image_path)
        segmentations = self.predict_segmentation(image_path)
        keypoints = self.predict_keypoints(image_path)
        
        annotations = {
            'image_path': image_path,
            'detections': detections,
            'segmentations': segmentations,
            'keypoints': keypoints,
            'num_detections': len(detections)
        }
        
        # Export format YOLO
        if save_annotations and detections:
            self.export_yolo_format(
                detections, 
                image_path, 
                config.ANNOTATIONS_DIR
            )
        
        return annotations
    
    def visualize_annotations(self, image_path: str, annotations: Dict) -> Image.Image:
        """Visualise les annotations sur l'image"""
        image = Image.open(image_path)
        img_array = np.array(image)
        
        fig, ax = plt.subplots(1, 1, figsize=(12, 12))
        ax.imshow(img_array)
        
        # Dessin des bboxes
        for det in annotations['detections']:
            bbox = det['bbox']
            img_h, img_w = img_array.shape[:2]
            
            # Conversion coordonnées normalisées → pixels
            x_center = bbox['x_center'] * img_w
            y_center = bbox['y_center'] * img_h
            width = bbox['width'] * img_w
            height = bbox['height'] * img_h
            
            x1 = x_center - width / 2
            y1 = y_center - height / 2
            
            rect = patches.Rectangle(
                (x1, y1), width, height,
                linewidth=2, edgecolor='red', facecolor='none'
            )
            ax.add_patch(rect)
            
            # Label
            ax.text(x1, y1 - 5, 
                   f"{det['class_name']} {det['confidence']:.2f}",
                   color='red', fontsize=10, weight='bold')
        
        ax.set_title(f"Annotations - {Path(image_path).name}")
        ax.axis('off')
        plt.tight_layout()
        return fig

# Initialisation
labeler = YOLOLabeler(config)

print("✅ Labellisateur YOLO initialisé!")
print("💡 Pour labelliser une image: annotations = labeler.label_image('path/to/image.png')")


In [ ]:
# ============================================================================
# STOCKAGE ANNOTATIONS SNOWFLAKE
# ============================================================================

def save_annotations_snowflake(plan_id: str, annotations: Dict):
    """Sauvegarde les annotations dans Snowflake"""
    if not snowflake_connected:
        print("⚠️ Snowflake non disponible, sauvegarde locale")
        # Sauvegarde locale JSON
        json_path = config.ANNOTATIONS_DIR / f"{plan_id}_annotations.json"
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(annotations, f, indent=2, ensure_ascii=False)
        print(f"💾 Annotations sauvegardées: {json_path}")
        return
    
    # Insertion des détections
    for det in annotations['detections']:
        query = f"""
        INSERT INTO annotations 
        (id, plan_id, classe, bbox_x, bbox_y, bbox_width, bbox_height, confidence)
        VALUES (
            '{plan_id}_det_{len(annotations['detections'])}',
            '{plan_id}',
            '{det['class_name']}',
            {det['bbox']['x_center']},
            {det['bbox']['y_center']},
            {det['bbox']['width']},
            {det['bbox']['height']},
            {det['confidence']}
        )
        """
        snowflake_mgr.execute_query(query, fetch=False)
    
    # Insertion des segmentations
    for seg in annotations['segmentations']:
        query = f"""
        INSERT INTO annotations 
        (id, plan_id, classe, segmentation_points, confidence)
        VALUES (
            '{plan_id}_seg_{len(annotations['segmentations'])}',
            '{plan_id}',
            '{seg['class_name']}',
            PARSE_JSON('{json.dumps(seg['segmentation_points'])}'),
            {seg['confidence']}
        )
        """
        snowflake_mgr.execute_query(query, fetch=False)
    
    print(f"✅ {len(annotations['detections'])} annotations sauvegardées dans Snowflake")

# Exemple d'utilisation
print("📝 Fonction de sauvegarde annotations prête")
print("💡 Utilisation: save_annotations_snowflake(plan_id, annotations)")


# Section 6: Entraînement YOLO

## 6.1 Préparation du dataset (train/val/test)


# Section 10: Exemples d'Utilisation Pratique

## 🎯 Exemples pour Démarrer Rapidement

Cette section contient des exemples prêts à l'emploi pour utiliser le projet.


## 10.1 Exemple Minimal: Traiter un PDF et Détecter les Éléments


In [ ]:
# ============================================================================
# EXEMPLE 1: TRAITEMENT COMPLET D'UN PDF
# ============================================================================

def exemple_traitement_pdf_complet(pdf_path: str):
    """
    Exemple complet: Extraction → Preprocessing → Détection → Visualisation
    
    Usage:
        exemple_traitement_pdf_complet("data/pdfs/mon_plan.pdf")
    """
    print("=" * 60)
    print("🚀 TRAITEMENT COMPLET D'UN PDF")
    print("=" * 60)
    
    # Étape 1: Extraction PDF → Images
    print("\n📄 Étape 1: Extraction des pages PDF...")
    result = process_pdf(pdf_path)
    
    if not result:
        print("❌ Erreur lors de l'extraction")
        return None
    
    print(f"✅ {len(result['images'])} pages extraites")
    
    # Étape 2: Preprocessing (première page)
    print("\n🔧 Étape 2: Preprocessing de la première page...")
    first_image_path = result['saved_paths'][0]
    processed_img = preprocess_image(
        first_image_path,
        save_path=str(config.IMAGES_DIR / f"processed_{Path(first_image_path).name}")
    )
    print("✅ Image préprocessée")
    
    # Étape 3: Détection avec YOLO
    print("\n🔍 Étape 3: Détection des éléments architecturaux...")
    annotations = labeler.label_image(first_image_path)
    
    print(f"✅ {len(annotations['detections'])} éléments détectés:")
    for det in annotations['detections']:
        print(f"   - {det['class_name']}: confiance {det['confidence']:.2%}")
    
    # Étape 4: Visualisation
    print("\n📊 Étape 4: Visualisation des résultats...")
    fig = labeler.visualize_annotations(first_image_path, annotations)
    plt.show()
    
    # Étape 5: Sauvegarde (optionnel)
    if snowflake_connected:
        print("\n💾 Étape 5: Sauvegarde dans Snowflake...")
        save_annotations_snowflake(result['plan_data']['id'], annotations)
    
    print("\n" + "=" * 60)
    print("✅ TRAITEMENT TERMINÉ AVEC SUCCÈS!")
    print("=" * 60)
    
    return {
        'result': result,
        'annotations': annotations,
        'processed_image': processed_img
    }

# DÉCOMMENTER POUR UTILISER:
# resultat = exemple_traitement_pdf_complet("data/pdfs/votre_plan.pdf")


## 10.2 Exemple: Traitement de Tous les PDFs d'un Dossier


In [ ]:
# ============================================================================
# EXEMPLE 2: TRAITEMENT BATCH DE PLUSIEURS PDFs
# ============================================================================

def exemple_traitement_batch(pdf_directory: str = "data/pdfs"):
    """
    Traite tous les PDFs d'un répertoire
    
    Usage:
        exemple_traitement_batch("data/pdfs")
    """
    pdf_dir = Path(pdf_directory)
    pdf_files = list(pdf_dir.glob("*.pdf"))
    
    if not pdf_files:
        print(f"❌ Aucun PDF trouvé dans {pdf_directory}")
        return []
    
    print(f"📚 {len(pdf_files)} PDFs trouvés")
    print("=" * 60)
    
    results = []
    
    for idx, pdf_path in enumerate(pdf_files, 1):
        print(f"\n📄 [{idx}/{len(pdf_files)}] Traitement: {pdf_path.name}")
        print("-" * 60)
        
        try:
            # Extraction
            result = process_pdf(str(pdf_path))
            
            if result:
                # Détection sur première page de chaque PDF
                if result['saved_paths']:
                    annotations = labeler.label_image(result['saved_paths'][0])
                    result['annotations'] = annotations
                    results.append(result)
                    
                    print(f"  ✅ {len(annotations['detections'])} détections")
                else:
                    print("  ⚠️ Aucune image extraite")
            else:
                print("  ❌ Erreur lors du traitement")
                
        except Exception as e:
            print(f"  ❌ Erreur: {e}")
    
    print("\n" + "=" * 60)
    print(f"✅ TRAITEMENT BATCH TERMINÉ: {len(results)}/{len(pdf_files)} PDFs traités")
    print("=" * 60)
    
    # Statistiques globales
    total_detections = sum(len(r.get('annotations', {}).get('detections', [])) for r in results)
    print(f"\n📊 Statistiques globales:")
    print(f"   - Total détections: {total_detections}")
    print(f"   - Moyenne par PDF: {total_detections/len(results) if results else 0:.1f}")
    
    return results

# DÉCOMMENTER POUR UTILISER:
# tous_les_resultats = exemple_traitement_batch()


## 10.3 Exemple: Détection et Export Vectoriel


In [ ]:
# ============================================================================
# EXEMPLE 3: DÉTECTION + EXPORT VECTORIEL
# ============================================================================

def exemple_detection_et_export(image_path: str, output_format: str = "dxf"):
    """
    Détecte les éléments et exporte en format vectoriel
    
    Args:
        image_path: Chemin vers l'image du plan
        output_format: "dxf" ou "ifc"
    
    Usage:
        exemple_detection_et_export("data/images/plan.png", "dxf")
    """
    print("=" * 60)
    print(f"🔍 DÉTECTION ET EXPORT {output_format.upper()}")
    print("=" * 60)
    
    # Détection complète
    print("\n1️⃣ Détection des éléments...")
    detections = labeler.predict_detection(image_path)
    segmentations = labeler.predict_segmentation(image_path)
    keypoints = labeler.predict_keypoints(image_path)
    
    annotations = {
        'detections': detections,
        'segmentations': segmentations,
        'keypoints': keypoints
    }
    
    print(f"   ✅ {len(detections)} détections")
    print(f"   ✅ {len(segmentations)} segmentations")
    print(f"   ✅ {len(keypoints)} keypoints")
    
    # Export (exemple simplifié - voir Section 7 pour l'implémentation complète)
    print(f"\n2️⃣ Export en format {output_format.upper()}...")
    
    if output_format.lower() == "dxf" and EZDXF_AVAILABLE:
        try:
            import ezdxf
            doc = ezdxf.new('R2010')
            msp = doc.modelspace()
            
            # Ajouter les éléments détectés comme lignes/polygones
            for det in detections:
                bbox = det['bbox']
                # Conversion coordonnées normalisées → pixels → mètres
                # (à adapter selon votre échelle)
                x = bbox['x_center'] * 1000  # Exemple: échelle
                y = bbox['y_center'] * 1000
                w = bbox['width'] * 1000
                h = bbox['height'] * 1000
                
                # Dessiner rectangle
                msp.add_lwpolyline([
                    (x - w/2, y - h/2),
                    (x + w/2, y - h/2),
                    (x + w/2, y + h/2),
                    (x - w/2, y + h/2),
                    (x - w/2, y - h/2)
                ])
            
            output_path = config.OUTPUT_DIR / f"{Path(image_path).stem}.dxf"
            doc.saveas(output_path)
            print(f"   ✅ Fichier sauvegardé: {output_path}")
            
        except Exception as e:
            print(f"   ❌ Erreur export DXF: {e}")
    
    elif output_format.lower() == "ifc" and IFC_AVAILABLE:
        print("   ⚠️ Export IFC - Voir Section 7 pour implémentation complète")
        # Implémentation IFC plus complexe, voir Section 7
    
    else:
        print(f"   ⚠️ Format {output_format} non disponible")
    
    # Visualisation
    print("\n3️⃣ Visualisation...")
    fig = labeler.visualize_annotations(image_path, annotations)
    plt.show()
    
    print("\n" + "=" * 60)
    print("✅ EXPORT TERMINÉ!")
    print("=" * 60)
    
    return annotations

# DÉCOMMENTER POUR UTILISER:
# annotations = exemple_detection_et_export("data/images/plan.png", "dxf")


## 10.4 Exemple: Dashboard Interactif avec Plotly


In [ ]:
# ============================================================================
# EXEMPLE 4: DASHBOARD INTERACTIF
# ============================================================================

def exemple_dashboard_interactif(image_path: str, annotations: Dict):
    """
    Crée un dashboard interactif avec Plotly
    
    Usage:
        annotations = labeler.label_image("data/images/plan.png")
        exemple_dashboard_interactif("data/images/plan.png", annotations)
    """
    print("📊 Création du dashboard interactif...")
    
    # Charger l'image
    img = Image.open(image_path)
    img_array = np.array(img)
    
    # Créer figure avec subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Plan Original', 'Détections', 
                       'Distribution des Classes', 'Confidences'),
        specs=[[{"type": "image"}, {"type": "image"}],
               [{"type": "bar"}, {"type": "histogram"}]]
    )
    
    # 1. Image originale
    fig.add_trace(
        go.Image(z=img_array),
        row=1, col=1
    )
    
    # 2. Image avec annotations
    # Dessiner les bboxes sur l'image
    annotated_img = img_array.copy()
    for det in annotations['detections']:
        bbox = det['bbox']
        h, w = img_array.shape[:2]
        
        x1 = int((bbox['x_center'] - bbox['width']/2) * w)
        y1 = int((bbox['y_center'] - bbox['height']/2) * h)
        x2 = int((bbox['x_center'] + bbox['width']/2) * w)
        y2 = int((bbox['y_center'] + bbox['height']/2) * h)
        
        # Dessiner rectangle (simplifié)
        annotated_img[y1:y1+3, x1:x2] = [255, 0, 0]  # Rouge
        annotated_img[y2-3:y2, x1:x2] = [255, 0, 0]
        annotated_img[y1:y2, x1:x1+3] = [255, 0, 0]
        annotated_img[y1:y2, x2-3:x2] = [255, 0, 0]
    
    fig.add_trace(
        go.Image(z=annotated_img),
        row=1, col=2
    )
    
    # 3. Distribution des classes
    classes = [det['class_name'] for det in annotations['detections']]
    class_counts = pd.Series(classes).value_counts()
    
    fig.add_trace(
        go.Bar(
            x=class_counts.index,
            y=class_counts.values,
            name="Classes"
        ),
        row=2, col=1
    )
    
    # 4. Histogramme des confidences
    confidences = [det['confidence'] for det in annotations['detections']]
    
    fig.add_trace(
        go.Histogram(
            x=confidences,
            nbinsx=20,
            name="Confidences"
        ),
        row=2, col=2
    )
    
    # Mise à jour layout
    fig.update_layout(
        height=800,
        title_text="Dashboard Interactif - Extraction de Plans",
        showlegend=False
    )
    
    # Afficher
    fig.show()
    
    print("✅ Dashboard créé et affiché!")

# DÉCOMMENTER POUR UTILISER:
# annotations = labeler.label_image("data/images/plan.png")
# exemple_dashboard_interactif("data/images/plan.png", annotations)


## 10.5 Résumé: Commandes Essentielles

### 🎯 Workflow Minimal (Copier-Coller)

```python
# 1. Setup (exécuter Section 1 d'abord)

# 2. Traiter un PDF
pdf_path = "data/pdfs/mon_plan.pdf"
result = process_pdf(pdf_path)

# 3. Détecter les éléments
annotations = labeler.label_image(result['saved_paths'][0])

# 4. Visualiser
labeler.visualize_annotations(result['saved_paths'][0], annotations)
plt.show()
```

### 📚 Pour Plus d'Informations

- **Guide complet**: Voir `GUIDE_UTILISATION.md`
- **Prompt expert**: Voir `PROMPT_EXPERT.md`
- **Documentation**: Chaque section du notebook contient des explications détaillées

---

## ✅ Checklist d'Utilisation

- [ ] Section 1 exécutée (Setup)
- [ ] PDFs placés dans `data/pdfs/`
- [ ] Section 3 exécutée (Extraction)
- [ ] Section 5 exécutée (Labellisation)
- [ ] Résultats visualisés
- [ ] (Optionnel) Section 4 exécutée (Génération données)
- [ ] (Optionnel) Section 6 exécutée (Entraînement)

---

**🎉 Vous êtes prêt à utiliser le projet !**


In [ ]:
# ============================================================================
# PRÉPARATION DATASET POUR ENTRAÎNEMENT YOLO
# ============================================================================

class DatasetPreparator:
    """Prépare le dataset au format YOLO"""
    
    def __init__(self, config):
        self.config = config
    
    def create_yolo_structure(self, base_dir: Path):
        """Crée la structure de répertoires YOLO"""
        dirs = {
            'train': ['images', 'labels'],
            'val': ['images', 'labels'],
            'test': ['images', 'labels']
        }
        
        for split, subdirs in dirs.items():
            for subdir in subdirs:
                (base_dir / split / subdir).mkdir(parents=True, exist_ok=True)
        
        print(f"✅ Structure YOLO créée dans {base_dir}")
    
    def split_dataset(self, image_paths: List[str], 
                     train_ratio: float = 0.7,
                     val_ratio: float = 0.2,
                     test_ratio: float = 0.1) -> Dict[str, List[str]]:
        """
        Divise le dataset en train/val/test
        
        Args:
            image_paths: Liste des chemins d'images
            train_ratio: Proportion train
            val_ratio: Proportion validation
            test_ratio: Proportion test
        """
        assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 0.01, "Ratios doivent sommer à 1"
        
        import random
        random.shuffle(image_paths)
        
        n_total = len(image_paths)
        n_train = int(n_total * train_ratio)
        n_val = int(n_total * val_ratio)
        
        splits = {
            'train': image_paths[:n_train],
            'val': image_paths[n_train:n_train + n_val],
            'test': image_paths[n_train + n_val:]
        }
        
        print(f"📊 Split dataset:")
        print(f"  Train: {len(splits['train'])} ({len(splits['train'])/n_total*100:.1f}%)")
        print(f"  Val: {len(splits['val'])} ({len(splits['val'])/n_total*100:.1f}%)")
        print(f"  Test: {len(splits['test'])} ({len(splits['test'])/n_total*100:.1f}%)")
        
        return splits
    
    def copy_to_yolo_structure(self, splits: Dict[str, List[str]], 
                               yolo_dir: Path, annotations_dir: Path):
        """Copie images et labels dans la structure YOLO"""
        import shutil
        
        for split, image_paths in splits.items():
            for img_path in image_paths:
                # Copie image
                img_name = Path(img_path).name
                dst_img = yolo_dir / split / "images" / img_name
                shutil.copy(img_path, dst_img)
                
                # Copie label (si existe)
                label_name = Path(img_path).stem + ".txt"
                src_label = annotations_dir / label_name
                if src_label.exists():
                    dst_label = yolo_dir / split / "labels" / label_name
                    shutil.copy(src_label, dst_label)
        
        print(f"✅ Dataset copié dans structure YOLO: {yolo_dir}")
    
    def create_yolo_config(self, yolo_dir: Path, num_classes: int) -> str:
        """Crée le fichier de configuration YOLO (data.yaml)"""
        class_names = [self.config.CLASSES[i] for i in sorted(self.config.CLASSES.keys())]
        
        config_content = f"""
# Dataset configuration for YOLO
path: {yolo_dir.absolute()}
train: train/images
val: val/images
test: test/images

# Classes
nc: {num_classes}
names:
"""
        for idx, name in enumerate(class_names):
            config_content += f"  {idx}: {name}\n"
        
        config_path = yolo_dir / "data.yaml"
        with open(config_path, 'w') as f:
            f.write(config_content)
        
        print(f"✅ Configuration YOLO créée: {config_path}")
        return str(config_path)

# Initialisation
dataset_prep = DatasetPreparator(config)

print("✅ Préparateur de dataset initialisé!")
print("💡 Pour préparer le dataset:")
print("   1. Créer structure: dataset_prep.create_yolo_structure(yolo_dir)")
print("   2. Split: splits = dataset_prep.split_dataset(image_paths)")
print("   3. Copier: dataset_prep.copy_to_yolo_structure(splits, yolo_dir, annotations_dir)")
print("   4. Config: dataset_prep.create_yolo_config(yolo_dir, num_classes)")
